### Finestre work in progress

![finestra esempio 1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/products-double-hung-window-2x.jpg?raw=true)
![finestra esempio 2](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/Custom-made-Aluminum-bay-window-for-sale.jpg?raw=true)


In [1]:
from pyplasm import *

Evaluating fenvs.py..
...fenvs.py imported in 0.004943 seconds


In [7]:
""" intRGBColor

Metodo preso dal web per ottenere colori da RGB
@param values: lista di tre interi RGB
@return: Colore
"""
def intRGBColor(values):
        return Color4f([ values[0]/255.0,
                         values[1]/255.0,
                         values[2]/255.0,
                         0.5])
GOLD=intRGBColor([238,232,170])
LIGHT_BLUE = intRGBColor([192,242,247])
LIGHT_BROWN = intRGBColor([193,157,98])

In [8]:
def create_openable_window(X,Y,Z,dx,dy,handle,mirror_or_not_mirror):
    sum=0
    toReturn=[]
    ehigth=0
    handle=COLOR(GOLD)(STRUCT([T(3)(Z[0]),R([1,2])(math.pi/2),handle]))
    for i in range(len(mirror_or_not_mirror)):
        for j in range(len(mirror_or_not_mirror[i])):
            if(mirror_or_not_mirror[i][j]==1):
                sum=sum+1
        if(i%2==0 and sum==(len(mirror_or_not_mirror[i])+2)):
            
            xscale=[]
            for scale in range(len(X)):
                if(scale%2==0):
                    xscale.append(X[scale])
                else:
                    xscale.append(X[scale]+dx)
            lenX=0
            for h in xscale:
                lenX= lenX +h
            if(lenX>0):
                toAdd=CUBOID([lenX,Y[i],Z[0]])
                toReturn.append(toAdd)
                toReturn.append(T(2)(Y[i]))
                toAdd2=CUBOID([Z[0],Y[i+1]+dy,Z[0]])
                glass=COLOR(LIGHT_BLUE)(STRUCT([T(3)(Z[0]/2.0),CUBOID([(lenX-1*Z[0]),Y[i+1]+dy,Z[0]/10.0])]))
                toAdd2=STRUCT([toAdd2,T(1)(Z[0]),glass,T(1)(lenX-2*Z[0]),toAdd2,T(1)(-(lenX-1*Z[0]))])
                toReturn.append(toAdd2)
                toReturn.append(T(2)(Y[i+1]+dy))
                toReturn.append(toAdd)
                toReturn.append(T(2)(Y[i]))
                toReturn=STRUCT(toReturn)
                toReturn=STRUCT([toReturn,T([1,2])([(SIZE(1)(toReturn))/2.0,(SIZE(2)(toReturn)+ehigth)-Z[0]]),handle])
                return toReturn
                
        elif(i%2==0 and i+1!=len(Y)):
            sum=0
            ehigth=ehigth+Y[i]+Y[i+1]+dy
            toReturn.append(T(2)(Y[i]))
            toReturn.append(T(2)(Y[i+1]+dy))
    return 0    

In [12]:
## <Design_of_Doors_and_Windows>
def design_4_Windows(X,Y,Z,mirror_or_not_mirror,recursive=False):

        
    def scale_and_rotate(dx,dy,dz):
        final_struct=[]
        sum=0
        window_and_handle=[]
        handle=STRUCT([CUBOID([Z[0]*2,Z[0],Z[0]]),T([1,3])([Z[0],Z[0]]),CUBOID([Z[0],Z[0]*5,Z[0]]),
                       T([1,2,3])([-Z[0],Z[0]*4,-Z[0]]),CUBOID([Z[0]*2,Z[0],Z[0]])])
        
        for i in range(len(mirror_or_not_mirror)):
            if(i==1):
                window_and_handle= create_openable_window(X,Y,Z,dx,dy,handle,mirror_or_not_mirror)
            for j in range(len(mirror_or_not_mirror[i])):
                scaleY=0 
                if (mirror_or_not_mirror[i][j]==0):
                    X[j]= (-1.0)*abs(X[j])
                else:
                    sum=sum+1
                    X[j]=abs(X[j])        
            
            if(recursive and sum==len(Y) and i==1):
                print("open the window")
                scaleY=dy
            
            elif(sum!=0):
                xscale=[]
                for scale in range(len(X)):
                    dx=abs(dx)
                    if(X[scale]<0):
                        dx=-1.0*dx
                    if(scale%2==0):
                        xscale.append(X[scale])
                    else:
                        xscale.append(X[scale]+dx)      
                if(i%2!=0):
                    scaleY=dy
                toAdd= PROD([QUOTE(xscale),QUOTE([Y[i]+scaleY])])
                final_struct.append(toAdd)
            final_struct.append(T(2)(Y[i]+scaleY))
            sum=0
        
        final_struct=STRUCT(final_struct)
        final_struct=(PROD([final_struct,QUOTE(Z)]))
        if(recursive==False):
            mirror=COLOR(LIGHT_BLUE)(inverte_0_1(X,Y,Z,(dx,dy,dz),mirror_or_not_mirror))
            final_struct=STRUCT([final_struct,T(3)(Z[0]/2.0),mirror])
            if((window_and_handle)!=0):
                final_struct=STRUCT([final_struct,T([2,3])([-Y[1]/3.0-dy,Z[0]]),window_and_handle])
    
        
        window1=final_struct
        window2=final_struct
        window3=final_struct 
        separator= CUBOID([Z[0]*3,SIZE(2)(window2),Z[0]]);
        
        window2=STRUCT([T([1,3])([Z[0],0]),window2])
        window2=(STRUCT([R([1,3])(math.pi*1/4),(window2)]))
        window3=STRUCT([R([1,3])(-math.pi*1/4),(window1)])
            
        value=math.sqrt(Z[0]*Z[0]*2)
        window3=STRUCT([T([1,3])([0,SIZE(3)(window3)-value]),window3])
        
        """k=(SIZE(1)(STRUCT([window2,window3])),SIZE(2)(STRUCT([window2,window3])),SIZE(3)(STRUCT([window2,window3])))
        if(recursive!=True):
            VIEW(STRUCT([SKEL_1(CUBOID([k[0],k[1],k[2]])),window2,window3]))"""
        
        
        window1=STRUCT([separator,T([1,3])([Z[0]*3,0]),window1,T(1)(SIZE(1)(window1)),separator])
        window1=STRUCT([window3,T([1])([SIZE(1)(window3)-Z[0],0]),window1])
        window1=STRUCT([T(3)(Z[0]),window1])
        window1=STRUCT([window1,T([1,3])([SIZE(1)(window1)-value/4.0,+value/4.0]),window2])
        
        if(recursive):
            return final_struct
        else:
            return window1

    return scale_and_rotate


    
def inverte_0_1(X,Y,Z,(dx,dy,dz),listOfList):
    for i in range(len(listOfList)):
        for j in range(len(listOfList[i])):
            if(listOfList[i][j]==0):
                listOfList[i][j]=1
            else:
                listOfList[i][j]=0
    if(i==0):
        return []

    valueZ=Z[0]
    return design_4_Windows(X,Y,[valueZ/10],listOfList,True)(dx,dy,dz)


In [13]:
def design_4_door(X,Y,Z,mirror_or_not_mirror,recursive=False):     
    def scale_door(dx,dy,dz):
        final_struct=[]
        sum=0
        handle=STRUCT([CUBOID([Z[0]*2,Z[0],Z[0]]),T([1,3])([Z[0],Z[0]]),CUBOID([Z[0],Z[0]*5,Z[0]]),
                       T([1,2,3])([-Z[0],Z[0]*4,-Z[0]]),CUBOID([Z[0]*2,Z[0],Z[0]])])
        
        for i in range(len(mirror_or_not_mirror)):
            for j in range(len(mirror_or_not_mirror[i])):
                scaleY=0 
                if (mirror_or_not_mirror[i][j]==0):
                    X[j]= (-1.0)*abs(X[j])
                else:
                    sum=sum+1
                    X[j]=abs(X[j])        
            
            if(sum!=0):
                xscale=[]
                for scale in range(len(X)):
                    dx=abs(dx)
                    if(X[scale]<0):
                        dx=-1.0*dx
                    if(scale%2==0):
                        xscale.append(X[scale])
                    else:
                        xscale.append(X[scale]+dx)      
                if(i%2!=0):
                    scaleY=dy
                toAdd= PROD([QUOTE(xscale),QUOTE([Y[i]+scaleY])])
                final_struct.append(toAdd)
            final_struct.append(T(2)(Y[i]+scaleY))
            sum=0
        
        final_struct=STRUCT(final_struct)
        final_struct=(PROD([final_struct,QUOTE(Z)]))
        if(recursive==False):
            mirror=COLOR(LIGHT_BLUE)(inverte_0_1(X,Y,Z,(dx,dy,dz),mirror_or_not_mirror))
            handle=COLOR(GOLD)(handle)
            final_struct=COLOR(LIGHT_BROWN)(final_struct)
            sizeOnY=SIZE(2)(final_struct)
            moveOnX=X[0]/2.0+Z[0]
            final_struct=STRUCT([final_struct,T(3)(Z[0]/2.0),mirror,T([1,2])([-moveOnX,sizeOnY/2.0]),handle])
            doorContainer=(SIZE(1)(final_struct),SIZE(2)(final_struct))
            doorContainer=OFFSET([Z[0],Z[0],Z[0]*3])(SKEL_1(CUBOID([doorContainer[0]+Z[0],doorContainer[1]+Z[0],0])))
            final_struct=STRUCT([doorContainer,T([1,2,3])([Z[0],Z[0],Z[0]]),final_struct])

        return final_struct


    return scale_door

In [ ]:
X=[1,5,1,5,1,5,1,5,1]
Y=[1,10,1,10,1,10,1]
Z=[0.5]
mirror_or_not_mirror=[
    [1,1,1,1,1,1,1,1,1],
    [1,0,0,0,0,0,0,0,1],
    [1,1,1,1,1,1,1,1,1],
    [1,0,0,0,0,0,0,0,1],
    [1,1,1,1,1,1,1,1,1],
    [1,0,1,0,1,0,1,0,1],
    [1,1,1,1,1,1,1,1,1]]

X1=[5,5,1,5,1,5,1]
Y1=[1,5,1,5,1,5,1,5,1,5,1]
Z1=[0.5]
mirror_or_not_mirror1=[
    [1,1,1,1,1,1,1],
    [1,0,1,0,1,0,1],
    [1,0,1,0,1,1,1],
    [1,0,1,0,1,0,1],
    [1,0,1,0,1,1,1],
    [1,0,1,0,1,0,1],
    [1,0,1,0,1,1,1],
    [1,0,1,0,1,0,1],
    [1,0,1,0,1,1,1],
    [1,0,1,0,1,0,1],
    [1,1,1,1,1,1,1]]
X2=[1.5,5,1,5,1]
Y2=[10,5,1,5,1,5,1,5,1]
Z2=[0.5]
mirror_or_not_mirror2=[
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1]]
VIEW(design_4_Windows(X,Y,Z,mirror_or_not_mirror)(1,1,1))
VIEW(design_4_door(X1,Y1,Z1,mirror_or_not_mirror1)(0.1,1,0.1))
VIEW(design_4_door(X2,Y2,Z2,mirror_or_not_mirror2)(0,0,0.1))



open the window
